In [15]:
from bs4 import BeautifulSoup
from requests import get
import json
import random
import string
import sqlite3
import logging
import re

In [14]:
# Input / Output
db_path = '../Data/moviereviews.db'

# Output
log_path = '../Logs/movieReviews.log'

logger = logging.getLogger()
fhandler = logging.FileHandler(filename=moviereviewLog_path, mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

# conn = sqlite3.connect(db_path)
# c = conn.cursor()

conn = sqlite3.connect('test.db')
c = conn.cursor()
c.execute(("""
        CREATE TABLE IF NOT EXISTS
            test(
                NODE TEXT,
                URL TEXT,
                REVIEWBODY TEXT,
                RATING TEXT,
                REVIEWRATING TEXT,
                BESTRATING TEXT,
                WORSTRATING TEXT,
                PRIMARY KEY (NODE, URL))
    """))

In [22]:
def generateNode(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node
    # print("Random alphanumeric String is:", result_str)
# generateNode(31)

In [46]:
host = 'https://www.imdb.com'
# https://www.imdb.com/search/title/?title_type=tv_movie,tv_episode&release_date=2000-01-01,2020-12-31&user_rating=1.0,10.0&languages=en&start=1&ref_=adv_nxt
# +50
# https://www.imdb.com/search/title/?title_type=tv_movie,tv_episode&release_date=2000-01-01,2020-12-31&user_rating=1.0,10.0&languages=en&start=51&ref_=adv_nxt
page_count = 0
movie_count = 0
review_count = 0
# umstellen auf 1001
for page in range (0,11):
    page_count= page*50+1
    # print(page_count)
    url = f'https://www.imdb.com/search/title/?title_type=tv_movie,tv_episode&release_date=2000-01-01,2020-12-31&user_rating=1.0,10.0&languages=en&start={page_count}&ref_=adv_nxt'
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
    # print(type(movie_containers))
    # print(len(movie_containers))
    i = 0
    for movie in movie_containers:
        # link to site with all review previews for each movie
        # link = 'https://www.imdb.com/' + movie_containers[i].a['href'] + 'reviews'
        link = "https://www.imdb.com/review/rw3878864/"
        # print(link)
        movie_count += 1
        i += 1
        review_collection = BeautifulSoup(get(link).text, 'html.parser').body.find_all('div', class_='actions text-muted')
        for tiles in review_collection:
            permalinks = tiles.find_all('a', href=True, text='Permalink')
            for permalink in permalinks:
                review_link = host + permalink['href']
                # print(review_link)
                review_count += 1
                review_soup = BeautifulSoup(get(review_link).text, 'html.parser')
                try:
                    warning = review_soup.body.find_all(string=re.compile('.*{0}.*'.format('Warning: Spoilers')), recursive=True)
                except:
                    pass
                container = review_soup.find("script",type="application/ld+json")
                for item in container:
                    if item.__contains__('http://schema.org'):
                        text = str(item.extract())
                        oJson = json.loads(text)
                        # print(oJson)
                        node = generateNode(31)
                        if (warning):
                            reviewBodyContainer = html_soup.body.find_all('div', class_="text show-more__control")
                            print(reviewBodyContainer)
                            # reviewBody = reviewBodyContainer[0].text
                        else:
                            reviewBody = str(oJson['reviewBody'])
                        reviewRating = str(oJson['reviewRating'])
                        worstRating = str(oJson['reviewRating']['worstRating'])
                        bestRating = str(oJson['reviewRating']['bestRating'])
                        ratingValue = str(oJson['reviewRating']['ratingValue'])
                        print('node:' + node)
                        print('url: '+ review_link)
                        print('reviewBody: ' + reviewBody)
                        print('worstRating: ' + worstRating)
                        print('bestRating: ' + bestRating)
                        print('ratingValue: ' + ratingValue)
                        # print("INSERT OR IGNORE INTO MOVIEREVIEWS (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);""",(node,url,reviewBody,reviewRating,ratingValue,bestRating,worstRating))
                        # c.execute("INSERT OR IGNORE INTO MOVIEREVIEWS (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,url,reviewBody,reviewRating,ratingValue,bestRating,worstRating))
                        # conn.commit()
print("number of reviews: " + str(review_count))


node:_:znodeFqTwvk73ZqQsRDvW4ORiIAf53qkcgoN
url: https://www.imdb.com/review/rw3878864/
reviewBody: Brooke Nevin & Robin Dunne star as Maggie and Mitch, who once upon a time were in college together and find themselves back in their hometown and begin to reacquaint their friendship. Maggie has only fond memories of Mitch, who helped her get home for Christmas when she was a freshman. Maggie always wondered what happened and why Mitch never came back to college... He also has no Christmas spirit whatsoever, so Maggie sets out to rekindle his spirit and hopefully some romance as well. She begins sending him anonymous packages full of Christmas cheer. To complicate matters the packages become a media frenzy and she is assigned to write a story on them for the local paper. Will Maggie tell Mitch before someone else takes credit for the cheer? Cute romantic Christmas flick.
worstRating: 1
bestRating: 10
ratingValue: 7
node:_:znodeNaEMlwPmf8bCoKYIWG9m3kaiteKiuJu
url: https://www.imdb.com/rev

KeyboardInterrupt: 

In [89]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)
movie_containers = html_soup.find_all('div', class_ = 'lister-item mode-advanced')
print(type(movie_containers))
print(len(movie_containers))
i = 0
link = 'https://www.imdb.com/' + movie_containers[0].a['href'] + 'reviews'
# print(link)

collection = get(link)
collection_soup = BeautifulSoup(collection.text, 'html.parser')
# print(reviews_soup)
# parsen bis ich zu dem permalink komme
# review_containers = reviews_soup.body.findAll('a', text=re.compile('^(<a href=")(.*)(">)Permalink(<\/a>)$'))
collection_containers = collection_soup.body.find_all('div', class_='actions text-muted')
# collection_containers = collection_soup.find_all('a', string='Permalink')
# review_containers = reviews_soup.find_all('a', href=True)
# for a in review_containers.find_all('a', href=True):
#     if a.text=='Permalink':
#         print(a['href'])

# print(collection_containers)
for address in collection_containers:
    b = address.find_all('a', href=True, text='Permalink')
    for c in b:
        print(host + c['href'])
    # a = b['href']
    print(b)
    # print(a)


<class 'bs4.element.ResultSet'>
50
https://www.imdb.com/review/rw3878864/
[<a href="/review/rw3878864/">Permalink</a>]
https://www.imdb.com/review/rw3754733/
[<a href="/review/rw3754733/">Permalink</a>]
https://www.imdb.com/review/rw5156128/
[<a href="/review/rw5156128/">Permalink</a>]
https://www.imdb.com/review/rw3612128/
[<a href="/review/rw3612128/">Permalink</a>]
https://www.imdb.com/review/rw5275443/
[<a href="/review/rw5275443/">Permalink</a>]
https://www.imdb.com/review/rw3597834/
[<a href="/review/rw3597834/">Permalink</a>]
https://www.imdb.com/review/rw5355667/
[<a href="/review/rw5355667/">Permalink</a>]
https://www.imdb.com/review/rw5774002/
[<a href="/review/rw5774002/">Permalink</a>]
https://www.imdb.com/review/rw3371112/
[<a href="/review/rw3371112/">Permalink</a>]


In [49]:
# snippets

print(type(container))

# response1 = get(a)
# microdata_content = response1.xpath('//script[@type="application/ld+json"]/text()').extract_first()
# microdata = json.loads(microdata_content)
#
# ratingValue = microdata["reviewRating"]["ratingValue"]
        # type(container)
        # str(container)
        #
# print(container)
for item in container:
    if item.__contains__('http://schema.org'):
        text = str(item.extract())
# print(text)
# print(type(text))
oJson = json.loads(text)
reviewbody = oJson['reviewBody']
print(type(reviewbody))
# results=[]
# for annotation in oJson:
#     results.append([annotation])
# df = pd.DataFrame(results)
# print(df)
        # print(type(container))
        # s1 = json.dumps(container)
        # data = json.loads(s1)
        # print(container)
        # print(data['reviewbody'])
        # data = json.loads()
    # print(collection_containers[0].find_all('a')[1]["href"])

    # print(collection_containers[0].find(string='Permalink').a['href'])
    # for movie in movie_containers:
    #     link = 'https://www.imdb.com/' + movie_containers[i].a['href']
    #     print(link)
    #     i += 1

    # print(movie_containers)
# get into each movie -> link
#     links = html_soup.find_all('a href', class_ = 'loadlate')
#     print(links)
# permalink


<class 'bs4.element.Tag'>


In [43]:
testurl = 'https://www.imdb.com/review/rw3288833/'
response = get(testurl)
html_soup = BeautifulSoup(response.text, 'html.parser')
# print(html_soup)
try:
    results = html_soup.body.find_all(string=re.compile('.*{0}.*'.format('Warning: Spoilers')), recursive=True)
except:
    pass
if (results):
    # print(results)
    # driver = webdriver.Chrome("C:\/Users\/D064966\/Downloads\/chromedriver")
    # driver.find_element_by_id('expander-icon-wrapper spoiler-warning__control').click()
    reviewBodyContainer = html_soup.body.find_all('div', class_="text show-more__control")
    reviewBody = reviewBodyContainer[0].text
    container = html_soup.find("script",type="application/ld+json")
    # print(container)
    for item in container:
        if item.__contains__('http://schema.org'):
            text = str(item.extract())
            oJson = json.loads(text)
            # print(oJson)
            node = generateNode(31)
            # reviewBody = str(oJson['reviewBody'])
            reviewRating = str(oJson['reviewRating'])
            worstRating = str(oJson['reviewRating']['worstRating'])
            bestRating = str(oJson['reviewRating']['bestRating'])
            ratingValue = str(oJson['reviewRating']['ratingValue'])
            print('node:' + node)
            print('url: '+ testurl)
            print('reviewBody: ' + reviewBody)
            print('worstRating: ' + worstRating)
            print('bestRating: ' + bestRating)
            print('ratingValue: ' + ratingValue)
else:
    pass
# review_collection = BeautifulSoup(html_soup).body.find_all('div', class_='actions text-muted')
# print(review_collection)
# print(html_soup)

node:_:znodeRzmbB24KtQ4P3QxmD50P9qprPPhRY9W
url: https://www.imdb.com/review/rw3288833/
reviewBody: Disney's Descendants is a miracle of campy film-making. The cheese levels are off the charts as the teenaged children of the Disney villains sing, dance, and fall in love with their heroic counterparts.The highlights have to be the so bad it's awful rap rendition of "Be Our Guest," the depiction of Maleficent as a tap dancing fiend, and the rest of the villains being depicted as washed up home-bodies who vegetate on the couch all day.To be fair to the younger actors, they're game with their performances. The script feels like a fan fiction written by middle school age kids with plot holes galore: how did the villains reproduce? I'm guessing they either asexually budded or hooked up with other villains to produce them? Also, if the nerdy Doug is the son of Dopey, then who's his mother? And this is just the icing on the cake: why is Prince Ben okay with Mal slipping him a love potion? How 